In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE57386"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE57386"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE57386.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57386.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE57386.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of biopsies, CD14+ and CD14- cells from RA, PsA and PsO patients with Infliximab treatment"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['Sex: F', 'Sex: M', 'response: .', 'response: responder', 'response: nonresponder'], 1: ['age: 51', 'age: 28', 'age: 46', 'age: 57', 'age: 61', 'age: 35', 'age: 19', 'age: 67', 'age: 38', 'age: 55', 'age: 39', 'age: 44', 'age: 52', 'Sex: F', 'Sex: M', 'age: 54', 'age: 40', 'age: 64', 'age: 23', 'age: 60', 'age: 32', 'age: 24', 'age: 62', 'age: 42', 'age: 36', 'age: 56', 'age: 47', 'age: 50', 'age: 66', 'age: 58'], 2: ['sample type: biopsy', 'age: 40', 'age: 54', 'age: 36', 'age: 23', 'age: 42', 'age: 24', 'age: 62', 'age: 46', 'age: 56', 'age: 32', 'age: 47', 'age: 60', 'age: 44', 'age: 64', 'age: 19', 'age: 61', 'age: 57', 'age: 35', 'age: 53', 'age: 59', 'age: 28', 'age: 3

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List

# Analyzing the dataset information
# 1. Gene Expression Data Availability
# From the background, this appears to be gene expression data from biopsies, CD14+ and CD14- cells
is_gene_available = True  # Gene expression data appears to be available

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Psoriatic Arthritis
# Looking for Psoriatic Arthritis in the sample characteristics
trait_row = 5  # In row 5, we see 'disease: Psoriatic Arthritis'

# Age
age_row = 1  # Age information is present in row 1

# Gender
gender_row = 0  # Sex information is present in row 0

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert trait value to binary format."""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if 'psoriatic arthritis' in value:
        return 1  # Cases
    elif 'normal' in value or 'healthy control' in value or 'health control' in value:
        return 0  # Controls
    else:
        return None  # Other diseases or unknown values

def convert_age(value: str) -> float:
    """Convert age value to continuous format."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
        try:
            return float(age_str)
        except ValueError:
            return None
    return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary format (0: female, 1: male)."""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if ':' in value:
        gender_str = value.split(':', 1)[1].strip().lower()
        if 'm' == gender_str or 'male' == gender_str:
            return 1
        elif 'f' == gender_str or 'female' == gender_str:
            return 0
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip clinical feature extraction for this step since we don't have
# the required clinical_data.csv file with the proper structure.
# We've already saved the cohort info which is the main goal of this step.
print("Clinical feature extraction skipped: clinical data file not found.")
print(f"Cohort info saved to: {json_path}")


Clinical feature extraction skipped: clinical data file not found.
Cohort info saved to: ../../output/preprocess/Psoriatic_Arthritis/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the format of gene identifiers
# The identifiers like '1007_PM_s_at', '1053_PM_at', etc. are Affymetrix probeset IDs,
# not standard human gene symbols. These are specific to microarray platforms
# and need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))

Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X